### 東京都文京区物件スクレイピング

参考文献
スクレイピング動画
- https://www.youtube.com/watch?v=l9KRgSUeNXk
- https://www.youtube.com/watch?v=gixuk9qFpRg
- https://www.youtube.com/watch?v=_H9bdFOZ4vs
- https://www.youtube.com/watch?v=Lq4xHWnT_KE

スプレッドシート動画
- https://youtu.be/fFSGPciIkfI?si=MQ4wrvIiFpmwgdCR

環境変数
- https://qiita.com/wooooo/items/7b57eaf32c22195df843

In [1]:
#ライブラリのインポート
from bs4 import BeautifulSoup
import re #不要文字削除
import requests
from time import sleep
from tqdm import tqdm   #for文の進捗確認
import pandas as pd

In [2]:
# 最後のページの数値を取得
url = 'https://suumo.jp/chintai/tokyo/sc_bunkyo/?page={}'
res = requests.get(url.format(1))
#1ページ目のページネーションに全ページの番号が表示される
#1ページ目のページネーション部分をスクレイピングし、その中の最後の要素（最後のページ番号）を取得
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
last_page = int(soup.find('ol', class_='pagination-parts').find_all('li')[-1].text)
#[-1]でページネーションのリストから最後の要素を取得
#textでページ番号を取得
#intで整数型に変換

In [3]:
#空のリストを作成
data_list = []

In [4]:
# 正常にHTML情報が取得できれば以下のコードを実行
if res.status_code == 200:

    #文京区のSUUMO掲載物件全ページ情報を取得
    for page in tqdm(range(1, last_page +1)): #for文の進捗を確認
    #range(start, stop) 関数は、startから始まってstop-1までの数値のシーケンスを生成
    #stopの値自体はシーケンスに含まれないため +1で最後のページもスクレイピング
        target_url = url.format(page)
        #ページ取得できているかの確認
        # print("data_listの大きさ:",len(data_list))
        # print(target_url)

        #requestを使ってURLにアクセス
        res = requests.get(target_url)
        #相手サイトの負荷軽減
        sleep(1)
        #文字化け防止
        res.encoding = 'utf-8'
        #取得したHTMLをBeautifulSoupで解析
        soup = BeautifulSoup(res.text, 'html.parser')

        #全ての物件情報取得
        contents = soup.find_all('div', class_= 'cassetteitem')

        #for文で物件・部屋情報取得
        for content in contents:
            #物件・部屋情報を解析
            detail = content.find('div', class_='cassetteitem-detail')
            table = content.find('table', class_='cassetteitem_other')

            #物件情報から必要情報を取得
            name = detail.find('div', class_='cassetteitem_content-title').text
            address = detail.find('li', class_='cassetteitem_detail-col1').text
            access = detail.find('li', class_='cassetteitem_detail-col2').text
            age, story = detail.find('li', class_='cassetteitem_detail-col3').text.split()

            #部屋情報を取得
            tr_tags = table.find_all('tr', class_='js-cassette_link')

            #for文で部屋情報取得
            for tr_tag in tr_tags:
                #部屋情報から必要情報を取得
                floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
                #さらに細かい情報取得
                rent, administration = price.find_all('li')
                deposit, gratuity = first_fee.find_all('li')
                madori, menseki = capacity.find_all('li')
                #取得した全ての情報を辞書に格納
                data = {
                    'name' : name,
                    'address' : address,
                    'access' : access,
                    'age' : age,
                    'story' : story,
                    'floor': floor.text,
                    'rent' : rent.text,
                    'administration' : administration.text,
                    'deposit' : deposit.text,
                    'gratuity' : gratuity.text,
                    'madori' : madori.text,
                    'menseki' : menseki.text
                }
                #取得した辞書を格納
                data_list.append(data)

  0%|          | 0/169 [00:00<?, ?it/s]

100%|██████████| 169/169 [05:31<00:00,  1.96s/it]


In [5]:
#最後のインデックスを確認
data_list[-1]

{'name': 'ＪＲ山手線 田端駅 2階建 築63年',
 'address': '東京都文京区本駒込４',
 'access': '\nＪＲ山手線/田端駅 歩13分\n都営三田線/千石駅 歩17分\n東京メトロ南北線/本駒込駅 歩18分\n',
 'age': '築63年',
 'story': '2階建',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t2階',
 'rent': '3万円',
 'administration': '2000円',
 'deposit': '-',
 'gratuity': '-',
 'madori': 'ワンルーム',
 'menseki': '8.2m2'}

In [6]:
#data_list の、access を分割して新しいキーを追加
for item in data_list:
    #先頭と末尾の改行文字を削除
    #strip() ・・・文字列の先頭と末尾にある空白文字を（スペース、タブ、改行文字（\n））を取り除く
    cleaned_access = item['access'].strip()
    # 改行で分割
    access_list = cleaned_access.split('\n')
    # 分割したデータを新しいキーとして追加
    item['access1'] = access_list[0] if len(access_list) > 0 else ""
    item['access2'] = access_list[1] if len(access_list) > 1 else ""
    item['access3'] = access_list[2] if len(access_list) > 2 else ""

### データフレーム格納

In [7]:
#データフレームを作成
df = pd.DataFrame(data_list)

In [8]:
#データ型の確認
df.dtypes

name              object
address           object
access            object
age               object
story             object
floor             object
rent              object
administration    object
deposit           object
gratuity          object
madori            object
menseki           object
access1           object
access2           object
access3           object
dtype: object

In [9]:
#データ確認
df.head()

,name,address,access,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1,access2,access3
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,\n東京メトロ丸ノ内線/後楽園駅 歩7分\nＪＲ中央線/飯田橋駅 歩11分\n都営三田線/春...,新築,15階建,\r\n\t\t\t\t\t\t\t\t\t\t\t7階,12.9万円,12000円,12.9万円,-,1DK,25.07m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,\n東京メトロ丸ノ内線/後楽園駅 歩7分\nＪＲ中央線/飯田橋駅 歩11分\n都営三田線/春...,新築,15階建,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,\n東京メトロ丸ノ内線/後楽園駅 歩7分\nＪＲ中央線/飯田橋駅 歩11分\n都営三田線/春...,新築,15階建,\r\n\t\t\t\t\t\t\t\t\t\t\t5階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,\n東京メトロ丸ノ内線/後楽園駅 歩7分\nＪＲ中央線/飯田橋駅 歩11分\n都営三田線/春...,新築,15階建,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,\n東京メトロ丸ノ内線/後楽園駅 歩7分\nＪＲ中央線/飯田橋駅 歩11分\n都営三田線/春...,新築,15階建,\r\n\t\t\t\t\t\t\t\t\t\t\t5階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分


### クレンジング

dfの全体の不要文字削除

In [10]:
#DB全体の不要な文字を取り除く
def remove_unwanted_chars(text):
    if isinstance(text, str):
        return re.sub('[\n\r\t]', '', text)
    return text

# データフレームのすべての要素に関数を適用
df = df.applymap(remove_unwanted_chars)

# 処理後のデータフレームの先頭５つを確認
df.head()

#df.applymap() : データフレーム全体に対して、カッコ内の関数を適応

,name,address,access,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1,access2,access3
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,東京メトロ丸ノ内線/後楽園駅 歩7分ＪＲ中央線/飯田橋駅 歩11分都営三田線/春日駅 歩7分,新築,15階建,7階,12.9万円,12000円,12.9万円,-,1DK,25.07m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,東京メトロ丸ノ内線/後楽園駅 歩7分ＪＲ中央線/飯田橋駅 歩11分都営三田線/春日駅 歩7分,新築,15階建,4階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,東京メトロ丸ノ内線/後楽園駅 歩7分ＪＲ中央線/飯田橋駅 歩11分都営三田線/春日駅 歩7分,新築,15階建,5階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,東京メトロ丸ノ内線/後楽園駅 歩7分ＪＲ中央線/飯田橋駅 歩11分都営三田線/春日駅 歩7分,新築,15階建,4階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,東京メトロ丸ノ内線/後楽園駅 歩7分ＪＲ中央線/飯田橋駅 歩11分都営三田線/春日駅 歩7分,新築,15階建,5階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分


accessの処理

In [11]:
#df から 'access' 列を削除
df = df.drop(columns=['access'])

#データ確認
df.head()

,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1,access2,access3
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,7階,12.9万円,12000円,12.9万円,-,1DK,25.07m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,4階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,5階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,4階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,5階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線/後楽園駅 歩7分,ＪＲ中央線/飯田橋駅 歩11分,都営三田線/春日駅 歩7分


In [12]:
#アクセスから路線・駅名・徒歩時間を抽出
df[['access1_line', 'access1_station', 'access1_walk']] = df['access1'].str.extract(r'(.+?)/(.+?) 歩(.+?)分')
df[['access2_line', 'access2_station', 'access2_walk']] = df['access2'].str.extract(r'(.+?)/(.+?) 歩(.+?)分')
df[['access3_line', 'access3_station', 'access3_walk']] = df['access3'].str.extract(r'(.+?)/(.+?) 歩(.+?)分')

# 徒歩時間の列を整数型に変換
df['access1_walk'] = pd.to_numeric(df['access1_walk'], errors='coerce').fillna(0).astype(int)
df['access2_walk'] = pd.to_numeric(df['access2_walk'], errors='coerce').fillna(0).astype(int)
df['access3_walk'] = pd.to_numeric(df['access3_walk'], errors='coerce').fillna(0).astype(int)

#df から 'access1,2,3' 列を削除
df = df.drop(columns=['access1','access2','access3'])

df.head()

,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1_line,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,7階,12.9万円,12000円,12.9万円,-,1DK,25.07m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,4階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,5階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,4階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,新築,15階建,5階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7


ageの処理

In [13]:
# age '新築' を '築0年' に、その後 '築' と '年' を削除
df['age'] = df['age'].str.replace('新築', '築0年').str.replace('築', '').str.replace('年', '')

# 文字列を数値に変換。変換できない値は NaN に置き換える
df['age'] = pd.to_numeric(df['age'], errors='coerce')

# NaN 値を 0 に置き換える
df['age'] = df['age'].fillna(-1).astype(int)

df.head()

,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1_line,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15階建,7階,12.9万円,12000円,12.9万円,-,1DK,25.07m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15階建,4階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15階建,5階,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15階建,4階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15階建,5階,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7


story, floorの処理

In [14]:
# story, floorのデータの表現を統一
df['story'] = df['story'].str.replace('地下', 'B').str.replace('地上', '-').str.replace('平屋', '1階建').str.replace('階建', 'F')
df['floor'] = df['floor'].str.replace('階', '')

df.head()

,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1_line,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,7,12.9万円,12000円,12.9万円,-,1DK,25.07m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,20.4万円,20000円,20.4万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,20.5万円,20000円,20.5万円,-,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7


日本円の処理

In [15]:
#お金（rent, administration	, deposit, gratuity）を整数型に変換
def yen_to_int(text):
    try:
        # エラーが出たので一旦全て文字列に変換
        text = str(text)

        if '-' in text:
            amount = 0
        elif '万円' in text:
            amount = float(text.replace('万円', '')) * 10000
        else:
            amount = float(text.replace('円', ''))
    except ValueError:
        amount = 0 # 形式に一致しない場合は0を返す
    return int(amount)

df['rent'] = df['rent'].apply(yen_to_int)
df['administration'] = df['administration'].apply(yen_to_int)
df['deposit'] = df['deposit'].apply(yen_to_int)
df['gratuity'] = df['gratuity'].apply(yen_to_int)


df.head()

,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1_line,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,7,129000,12000,129000,0,1DK,25.07m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,204000,20000,204000,0,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,204000,20000,204000,0,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,205000,20000,205000,0,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,205000,20000,205000,0,2LDK,40.48m2,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7


mensekiの処理

In [16]:
#mensekiのデータ（比例尺度）に変換
df['menseki'] = df['menseki'].str.replace('m2','').astype(float)

df.head()

,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1_line,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,7,129000,12000,129000,0,1DK,25.07,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,204000,20000,204000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,204000,20000,204000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,205000,20000,205000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,205000,20000,205000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7


### 重複削除

In [17]:
# 特定の列に基づいて重複を確認
duplicate_rows = df[df.duplicated(subset=['address', 'age', 'floor', 'rent', 'menseki'])]

# 重複件数の表示
print(f"重複件数: {duplicate_rows.shape[0]}")

# 重複データの表示
print(duplicate_rows)


重複件数: 2389
                         name     address  age story floor    rent  \
6               クレヴィアリグゼ文京後楽園   東京都文京区春日１    0   15F     4  204000   
7               クレヴィアリグゼ文京後楽園   東京都文京区春日１    0   15F     4  205000   
8               クレヴィアリグゼ文京後楽園   東京都文京区春日１    0   15F     5  205000   
9               クレヴィアリグゼ文京後楽園   東京都文京区春日１    0   15F    12  211000   
18     東京メトロ南北線 東大前駅 9階建 築13年   東京都文京区西片２   13    9F     7  149000   
...                       ...         ...  ...   ...   ...     ...   
6638  東京メトロ南北線 東大前駅 12階建 築39年   東京都文京区白山１   39   12F     7   68000   
6644       ＪＲ山手線 池袋駅 2階建 築61年  東京都文京区目白台２   61    2F     1   73000   
6645       ＪＲ山手線 田端駅 2階建 築63年  東京都文京区本駒込４   63    2F     1   30000   
6646       ＪＲ山手線 田端駅 2階建 築63年  東京都文京区本駒込４   63    2F     2   30000   
6647       ＪＲ山手線 田端駅 2階建 築63年  東京都文京区本駒込４   63    2F     2   30000   

      administration  deposit  gratuity madori  menseki access1_line  \
6              20000   204000         0   2LDK    40.48     東京メトロ南北線   
7   

In [18]:
# 重複データの削除
df.drop_duplicates(subset=['address', 'age', 'floor', 'rent', 'menseki'], inplace=True)

CSV出力

In [19]:
df.to_csv('SUUMO_bunkyo2.csv', index=False, encoding='utf-8-sig')

print("csv出力が完了しました")

#df.applymap() : データフレーム全体に対して、カッコ内の関数を適応
#lambda x: re.sub('\n', ' ', x)) : 引数で渡された`x`に対して、`\n`を半角スペースに置換

df.head()

csv出力が完了しました


,name,address,age,story,floor,rent,administration,deposit,gratuity,madori,menseki,access1_line,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk
0,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,7,129000,12000,129000,0,1DK,25.07,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
1,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,204000,20000,204000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
2,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,204000,20000,204000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
3,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,4,205000,20000,205000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7
4,東京メトロ丸ノ内線 後楽園駅 15階建 新築,東京都文京区春日１,0,15F,5,205000,20000,205000,0,2LDK,40.48,東京メトロ丸ノ内線,後楽園駅,7,ＪＲ中央線,飯田橋駅,11,都営三田線,春日駅,7


In [20]:
#dfの大きさ確認
df.shape

(4259, 20)

### スプレッドシートに出力

In [21]:
#google spread sheets 出力
#ライブラリのインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#環境変数関連
from dotenv import load_dotenv
load_dotenv()
import os

In [22]:
#スコープとjsonファイルを使って認証情報を取得
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)

In [23]:
#認証情報をauthorize関数に渡してスプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

#シート情報を取得して変数に代入
SPREADSHEET_KEY = os.getenv('SPREADSHEET_KEY')
workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("suumo_DB2")

In [24]:
# NaN 値や無限大の値を None に置き換える
df = df.where(pd.notnull(df), None)
# データフレーム内に NaN 値や無限大の値があった（？）ためJSONでは扱えないとエラーに。
# JSONでも扱える形式に変換

# dfから値を習得
values = [df.columns.values.tolist()] + df.values.tolist()

# ワークシートの指定したセル(B2)から値を追加
worksheet.update("B2", values)


/var/folders/3d/lpwdp4ts3r5grm10hjnwtbnm0000gn/T/ipykernel_81142/631865491.py:10: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("B2", values)


{'spreadsheetId': '14JE9_LppdTkFaE533XHqdfOmKSLTas24s_j8RGP_u8A',
 'updatedRange': 'suumo_DB2!B2:U4261',
 'updatedRows': 4260,
 'updatedColumns': 20,
 'updatedCells': 84718}

### SQLite

In [25]:
#ライブラリのインポート
import sqlite3

In [26]:
# SQLiteデータベースに接続
#SQLのデータベースの枠を作成
db_name = "scraping_fudosan.db"
conn = sqlite3.connect(db_name)

In [27]:
#作成できているか確認
conn

In [28]:
#dfをデータベースに入れ込む
df.to_sql("suumo_data", conn, if_exists="replace", index=False)

4259

In [29]:
# データベース接続を閉じる
conn.close()

utf-8, utf-8-sig の違い
https://qiita.com/showmurai/items/60d32006d13512ffeaff

エンコードのコーデックに 'utf-8-sig' とすると、BOM有りの場合スキップして読み込んでくれる。
BOM無しの場合はそのままUTF-8として読み込むことが可能。

UTF-8 には BOM(Byte order mark)が付く場合がある。
UTF-8にBOMを付けるのは、Windows'メモ帳'やExcel。LinuxやMacは基本的にBOM無しUTF-8で扱う。

スクレイピングエラーが発生する場合考えられる事項。

以下、調べた結果
- ウェブサイトのアンチスクレイピング機能
    - 多くのウェブサイトは、スクレイピングを検出してブロックするための機能を備えている。頻繁なアクセスや一定のパターンのアクセスが検出されると、IPアドレスが一時的にブロックされることがある。

- リクエストの制限
    - サーバーは一定時間内のリクエスト数に制限を設けている場合がある。これに達すると、それ以上のリクエストは拒否されるか、エラーが返される。

- タイムアウトエラー
    - ネットワークの問題やサーバーの過負荷などにより、リクエストがタイムアウトすることがある。

- ページ構造の変更
    - スクレイピングしているウェブサイトがページの構造を変更した場合、コードが正しく機能しなくなることがある。

- スクリプトのエラー
    - スクリプト自体に問題がある場合。例えば、特定の条件下でエラーが発生する可能性があるコードなど。

一般的には、ウェブサイトやサーバーのポリシーによって、一度のセッションで取得できるデータ量やリクエスト数に制限が設けられることがある。